In [4]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Implementing KNN model

#### preparing Data

In [5]:
#this is the path of the dataset
path = '/Users/syedfaquaruddinquadri/Documents/Applied ML and DM/Final Project/machine-learning-dse-i210-final-project-yelprestaurantrecommender/data/processed'

In [6]:
#reading the business data into pandas data frame
nola_business_df = pd.read_csv(path+"/business.csv")

#dropping the columns we dont need
nola_business_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

#filtering only the restaurants business
rest_business = nola_business_df[nola_business_df['categories'].str.contains('Restaurants')==True]

#selecting features only required
nola_rest = rest_business[['attributes', 'categories', 'business_id', 'review_count', 'stars', 'name']]

In [7]:
#reading the reviews csv into dataframe
reviews_path = '/Users/syedfaquaruddinquadri/Documents/Applied ML and DM/Final Project'
reviews = pd.read_csv(reviews_path+"/reviews.csv")

#dropping unneccessary features
reviews.drop(['Unnamed: 0', 'cool', 'date', 'funny', 'useful'], 
             axis=1, 
             inplace=True)

/var/folders/6y/65h__9y57mg1xqznxbt3fvtr0000gn/T/ipykernel_93365/1063384312.py:3: DtypeWarning: Columns (0,1,2,3,5,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv(reviews_path+"/reviews.csv")


In [8]:
#merging two dataframes on business ids
rev = pd.merge(nola_rest, 
         reviews,left_on='business_id', 
         right_on='business_id', how = 'inner')

In [9]:
#renaming the columns
rev.rename(columns = {'stars_x':'rest_rating', 'stars_y':'user_rating', 'text':'text_review'}, inplace = True)

In [10]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error

#### preparing item-user matrix

In [40]:
%%time
# Creating an item-user interaction matrix
interaction_matrix = rev.pivot_table(index='business_id', columns='user_id', values='rest_rating', fill_value=0)

CPU times: user 22.5 s, sys: 15.9 s, total: 38.4 s
Wall time: 48 s


In [12]:
interaction_matrix

user_id,---UgP94gokyCDuB5zUssA,--1orhUoGFSdHXsoxqQc8g,--3PldvWEZ_bhLNsyrAtgA,--3WaS23LcIXtxyFULJHTA,--48iehfEvtnSrLq8grJIQ,--4wsx_CrQyZxJYvHjUuaw,--5IPONUyyQt38e85P1KdQ,--5VfF9MeCYnNiGFp3BXpw,--78aksX3obHJ6671pjJqA,--CIuK7sUpaNzalLAlHJKA,...,zzgLgszu48af4NJULymzAA,zzgY5GgMSc7Iin2W9oqd6w,zziWJMYwDjyVi7kJmgRUvg,zzisaH1aDfNlCTenS5co9w,zzlZ2oy-hOGXQGqDMF0IFg,zzq6Nhcq0JMbm7ExFFCP2A,zzqK0h_3FLhcKbX5H6rDZA,zzsqjDvanJhH9tn8NautOQ,zzuIXWg1WRhQXheMIdGZ5w,zzxZoMmjbUjXcWZzrE3PIw
business_id,,,,,,,,,,,,,,,,,,,,,
-0__F9fnKt8uioCKztF5Ww,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-0ltw8--HLuulPyOSspqAQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-1XSzguS6XLN-V6MVZMg2A,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-4x3pVUUsfWmKEilWKsOZQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-8B29WNRCyg4jViK4ChKlQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zvGNZF827KyzLupKiG4Xtw,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
zwe9H6Xxqb1_E09A20Ptgg,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
zxfqXVgZ_NvLY2lc1DAlJQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0


In [13]:
%%time
# Converting the interaction matrix to a sparse matrix
sparse_matrix = csr_matrix(interaction_matrix.values)

CPU times: user 2min 20s, sys: 17min 6s, total: 19min 27s
Wall time: 27min 30s


#### preparing KNN model

In [14]:
# Create and fit the kNN model
k = 5
model = NearestNeighbors(n_neighbors=k, metric='cosine', algorithm='brute')
model.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
user_id = 'nn6DoANEtr7SgvWWgrh2oQ'
user_idx = interaction_matrix.columns.get_loc(user_id)

# Getting the rating for each business the user has reviewed
user_ratings = interaction_matrix.iloc[:, user_idx]

#geting the mean rating for each business the user has reviewd
mean_user_rating = np.mean(user_ratings[user_ratings > 0])

In [16]:
# Predict ratings for all businesses using kNN
distances, indices = model.kneighbors(sparse_matrix, n_neighbors=k+1)
predicted_ratings = np.array([np.mean(interaction_matrix.iloc[indices[row_idx], user_idx]) 
                              for row_idx in range(interaction_matrix.shape[0])])

In [17]:
# Adjust the predicted ratings by adding the mean user rating
adjusted_ratings = predicted_ratings + mean_user_rating

In [18]:
# Get the indices of the top 5 recommended businesses
num_recommendations = 5
top_business_indices = adjusted_ratings.argsort()[-num_recommendations:][::-1]


In [19]:
# Get the business IDs of the top recommended businesses
top_business_ids = interaction_matrix.index[top_business_indices].tolist()

In [20]:
len(top_business_ids)

5

In [21]:
# selecting rows based on condition
recommended_restaurants = nola_rest.loc[nola_rest['business_id'].isin(top_business_ids)][['business_id', 'name']]

In [22]:
recommended_restaurants

,business_id,name
3,YNjyv0gfOr2g8lbmUpTnKg,Copper Vine
1284,el6K1DsZRnKNNaZ14YGMHQ,Salads Galore & More
1627,fUUNiG5x6wP84ty5GoqtRw,The Neutral Ground Bar & Grill
3451,UEPf0MGsw2mOlarskQX1Zg,Streetcar Poboys & Seafood
4881,ISm3ZCdmYsxwy7c8sBm1gw,Thaihey Nola


In [23]:
from sklearn.neighbors import NearestNeighbors
# Create and fit the kNN model
k = 5
model = NearestNeighbors(n_neighbors=k, 
                         metric='cosine', 
                         algorithm='brute')
model.fit(sparse_matrix)

# Function to make recommendations based on the nearest neighbors of an item
def recommend_restaurants(user_id, num_recommendations=5):
    user_idx = interaction_matrix.columns.get_loc(user_id)

    # Getting the rating for each business the user has reviewed
    user_ratings = interaction_matrix.iloc[:, user_idx]

    #geting the mean rating for each business the user has reviewd
    mean_user_rating = np.mean(user_ratings[user_ratings > 0])

    # Predict ratings for all businesses using kNN
    distances, indices = model.kneighbors(sparse_matrix, n_neighbors=k+1)
    predicted_ratings = np.array([np.mean(interaction_matrix.iloc[indices[row_idx], user_idx]) for row_idx in range(interaction_matrix.shape[0])])

    # Adjust the predicted ratings by adding the mean user rating
    adjusted_ratings = predicted_ratings + mean_user_rating

    # Get the indices of the top recommended businesses
    top_business_indices = adjusted_ratings.argsort()[-num_recommendations:][::-1]

    # Get the business IDs of the top recommended businesses
    top_business_ids = interaction_matrix.index[top_business_indices].tolist()
    
    recommended_restaurants = nola_rest.loc[nola_rest['business_id'].isin(top_business_ids)][['business_id', 'name']]


    return recommended_restaurants

# Example usage: Recommend restaurants for a specific user
user_id = 'nn6DoANEtr7SgvWWgrh2oQ'
recommended_restaurants = recommend_restaurants(user_id)
print(recommended_restaurants)




                 business_id                            name
3     YNjyv0gfOr2g8lbmUpTnKg                     Copper Vine
1284  el6K1DsZRnKNNaZ14YGMHQ            Salads Galore & More
1627  fUUNiG5x6wP84ty5GoqtRw  The Neutral Ground Bar & Grill
3451  UEPf0MGsw2mOlarskQX1Zg      Streetcar Poboys & Seafood
4881  ISm3ZCdmYsxwy7c8sBm1gw                    Thaihey Nola


In [25]:

from sklearn.neighbors import NearestNeighbors

# Create and fit the kNN model
k = 32
model = NearestNeighbors(n_neighbors=k, 
                         metric='cosine', 
                         algorithm='brute')
model.fit(sparse_matrix)

# Function to make recommendations based on the nearest neighbors of an item
def recommend_restaurants(user_id, num_recommendations=5):
    user_idx = interaction_matrix.columns.get_loc(user_id)

    # Getting the rating for each business the user has reviewed
    user_ratings = interaction_matrix.iloc[:, user_idx]

    #geting the mean rating for each business the user has reviewd
    mean_user_rating = np.mean(user_ratings[user_ratings > 0])

    # Predict ratings for all businesses using kNN
    distances, indices = model.kneighbors(sparse_matrix, n_neighbors=k+1)
    predicted_ratings = np.array([np.mean(interaction_matrix.iloc[indices[row_idx], user_idx]) for row_idx in range(interaction_matrix.shape[0])])

    # Adjust the predicted ratings by adding the mean user rating
    adjusted_ratings = predicted_ratings + mean_user_rating

    # Get the indices of the top recommended businesses
    top_business_indices = adjusted_ratings.argsort()[-num_recommendations:][::-1]

    # Get the business IDs of the top recommended businesses
    top_business_ids = interaction_matrix.index[top_business_indices].tolist()
    
    recommended_restaurants = nola_rest.loc[nola_rest['business_id'].isin(top_business_ids)][['business_id', 'name']]


    return recommended_restaurants

# Example usage: Recommend restaurants for a specific user
user_id = 'wDmVI9I07DniwYAz8_cBPA'
recommended_restaurants = recommend_restaurants(user_id)
print(recommended_restaurants)


                 business_id                    name
1323  2r3-fBAvSsljWrDLXkiHiw         LemonShark Poke
1999  hay0mXRcirAgdKWnBblidQ           Av Tree Farms
3104  dIQGbalj-3I8vixZ4M149g  Molly's Rise and Shine
3224  WYSqcpQv0OA8RyE7f2mnnw           Effervescence
5357  PRrWtQ3rdINngwESZNefVQ              Catalino's
